# This is a Jupyter notebook for testing how well the [NER model](https://huggingface.co/dbmdz/flair-historic-ner-onb) for historic German performs on our Travelogues corpus.
## Please note the comments that are given in each cell.


At first, please try downloading the flair and other packages that are needed for using this model.

In [92]:
import glob
import pprint
import re
import json

from flair.data import Sentence
from flair.models import SequenceTagger
import spacy
from spacy.tokens import Doc

from typing import List

from correct_ocr import single_characters, delete_specials, correct_s

In [76]:
METADATA_PATH: str = '../data/metadata/'
VALUES: List[str] = ['Title', 'Contributor']

The following cell is only needed if problems with spacy.load occur.

In [ ]:
import spacy.cli
spacy.cli.download("de_core_news_md")

This part will try out the NER on sentences from the Travelogues texts.

In [11]:
# Downloading language model for the spacy pipeline
nlp = spacy.load("de_core_news_md")

# Read in files – can be noisy OCR
file: str = open('../data/test/bossmann_gvinea_1708.txt', 'r').read()[100:2000]

# Corrections as implemented by @Lisa Braune

file = re.sub('aͤ', 'ä', file)
file = re.sub('uͤ', 'ü', file)
file = re.sub('oͤ', 'ö', file)

file = correct_s(file)
file = single_characters(file)
file = delete_specials(file)



# Throw document into spacy pipeline, sentencise file
doc: Doc = nlp(file)
sents: list = [sent.text for sent in doc.sents]

tagger: SequenceTagger = SequenceTagger.load("dbmdz/flair-historic-ner-onb")

for sent in sents:
    # Transform each sentence in the list into type Sentence for function availability
    sentence = Sentence(sent)
    tagger.predict(sentence)
    print(sentence.to_tagged_string())


/Users/sarahreb/.conda/envs/projektseminar_22-23_dh_trier/lib/python3.8/site-packages/huggingface_hub/file_download.py:594: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-12-02 13:09:47,020 loading file /Users/sarahreb/.flair/models/flair-historic-ner-onb/63111d37e8f19b08b01200ec38cd2b093d72026e56bbe99a7b25b6e3f8b7da8d.d53b1d9a206921442955a318ba5bbef2af5aabb93c4713d1ed3b8fe8c28cda3f
2022-12-02 13:09:48,957 SequenceTagger predicts: Dictionary with 16 tags: <unk>, O, S-PER, S-LOC, B-PER, E-PER, S-ORG, B-LOC, E-LOC, I-PER, B-ORG, E-ORG, I-LOC, I-ORG, <START>, <STOP>
Sentence: "0009 Reyse nach GVINEA , oder ausführliche Beschreibung dasiger GoldGruben ElephantenZähn und SclavenHandels nebst derer Einwohner" → ["GVINEA"/LOC]
Sentence: "Sitten Religion"
Sentence: "Regiment"
Sentence: "Kriegen"
Sentence: "Heyrathen und Begräbnissen"
Sentence: "auch allen hieselbst befindlichen Thieren"
Sentence: "so bishero in Europa unbekandt gewesen ." → ["Europa"/LOC]
Sentence: "Jm Frantzösischen herausgegeben durch Wilhelm Boßmann gewesenen Rahtsherrn OberKauffmann und Landes UnterCommandeur von der HolländischOst-Jndischen Compagnie ." → ["Wilhelm Boßmann"/PER, "Holl

In [ ]:
sentence = Sentence(file)
tagger.predict(sentence)
print(sentence.to_dict())


#### NER on Travelogues titles

In [122]:
def read_jsons(metadata_path: str, single_file: bool = False, doc_barcode: str = 'Z124117102', values: List[str] = 'Title') -> str:
    """
    Function that aims to retrieve certain metadata information from the JSON files. Can handle single and multiple file attempts.
    :param metadata_path: Path for metadata files.
    :param single_file: Boolean, False if all files in metadata directory should be observed, True if only a single file should be parsed.
    :param doc_barcode: If a single file is parsed, the barcode must be provided.
    :param values: List of tags for the information value someone wants to extract, e.g. the title or author of a file.
    :return: String of the metadata information.
    """
    if single_file:
        indicator = doc_barcode
    else:
        indicator = '*'

    for metadata_file in glob.glob(metadata_path + indicator + '.json')[:10]:
        with open(metadata_file, 'r') as f:
            object_dict = json.load(f)

            for metadata_dict in object_dict:
                try:
                    if metadata_dict['label'][0]['@value'] in values:
                        print(f"{metadata_dict['label'][0]['@value']}: {metadata_dict['value']}")
                except TypeError:
                    if metadata_dict['label'] in values:
                        print(f"{metadata_dict['label']}: {metadata_dict['value']}")

In [124]:
read_jsons(metadata_path=METADATA_PATH, values=VALUES, single_file=True)

Contributor: <a href='http://d-nb.info/gnd/129868868'>Bosman, Willem, 1672-</a> - Heyl
Title: Reyse nach Guinea, oder ausführliche Beschreibung dasiger Gold-Gruben ... Im Frantzösischen herausgegeben durch Wilhelm Boßmann ... Nun aber ins Hochteutsche übersetzet, und mit Kupffern gezieret
